In [1]:
#Importing packages
from selenium import webdriver
import pandas as pd

In [16]:
#change directory where you put your chrome driver
options = webdriver.ChromeOptions()
options.add_argument('headless')
driver = webdriver.Chrome("chromedriver", options=options)

In [25]:
%%time
all_comments = pd.DataFrame(columns = ['Date','user_id', 'comments']) 

# I already scraped p334 to p434
# now I'am scraping page 284 to 333
x = 284
pages_scraped = 333
driver.get('https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans/p{}'.format(x))

while x <= pages_scraped:
    print("page",x)
    
    if x > 1:
        driver.find_element_by_link_text("{}".format(x)).click()
        driver.implicitly_wait(3)
    
    ids = driver.find_elements_by_xpath("//*[contains(@id,'Comment_')]")
    comment_ids = []
    for i in ids:
        comment_ids.append(i.get_attribute('id'))

    for d in comment_ids:

        #Extract dates from for each user on a page
        user_date = driver.find_elements_by_xpath('//*[@id="' + d +'"]/div/div[2]/div[2]/span[1]/a/time')[0]
        date = user_date.get_attribute('title')

        #Extract user ids from each user on a page
        userid_element = driver.find_elements_by_xpath('//*[@id="' + d +'"]/div/div[2]/div[1]/span[1]/a[2]')[0]
        userid = userid_element.text

        #Extract Message for each user on a page
        if driver.find_elements_by_xpath('//*[@id="' + d +'"]/div/div[3]/div/div[1]/p'):
            user_message = driver.find_elements_by_xpath('//*[@id="' + d +'"]/div/div[3]/div/div[1]/p')[0]
            comment = user_message.text
        elif driver.find_elements_by_xpath('//*[@id="' + d +'"]/div/div[3]/div/div[1]/ul/li'):
            user_message = driver.find_elements_by_xpath('//*[@id="' + d +'"]/div/div[3]/div/div[1]/ul/li')[0]
            comment = user_message.text
        elif driver.find_elements_by_xpath('//*[@id="' + d +'"]/div/div[3]/div/div[1]'):
            user_message = driver.find_elements_by_xpath('//*[@id="' + d +'"]/div/div[3]/div/div[1]')[0]
            comment = user_message.text
        else: 
            comment = None
        
        #Adding date, userid and comment for each user in a dataframe    
        all_comments.loc[len(all_comments)] = [date,userid,comment]
        
    x += 1

page 284
page 285
page 286
page 287
page 288
page 289
page 290
page 291
page 292
page 293
page 294
page 295
page 296
page 297
page 298
page 299
page 300
page 301
page 302
page 303
page 304
page 305
page 306
page 307
page 308
page 309
page 310
page 311
page 312
page 313
page 314
page 315
page 316
page 317
page 318
page 319
page 320
page 321
page 322
page 323
page 324
page 325
page 326
page 327
page 328
page 329
page 330
page 331
page 332
page 333
Wall time: 4h 33min 21s


In [26]:
all_comments.shape

(2500, 3)

In [27]:
# all_comments.to_csv("2500_comments_edmunds.csv", sep="\t")

In [69]:
# all_comments.to_csv("5000_comments_edmunds.csv", sep="\t")

#### check the data

In [29]:
import pandas as pd
df = pd.read_csv("5000_comments_edmunds.csv", '\t', parse_dates=['Date'])

In [77]:
len(df['user_id'].value_counts())

68

In [30]:
df.head()

,Unnamed: 0,Date,user_id,comments
0,0,2014-02-20 14:06:00,robr2,Someone better call marketing then:
1,1,2014-02-20 14:14:00,andys120,All what? This whole renaming scheme has turne...
2,2,2014-02-20 14:34:00,flightnurse,LOL... Oh well....
3,3,2014-02-20 15:04:00,Michaell,"Isn't there going to be a 4-series ""grand coup..."
4,4,2014-02-20 15:13:00,kyfdx,l


In [31]:
df2 = pd.read_csv("2500_comments_edmunds.csv", '\t', parse_dates=['Date'])

In [32]:
df3 = pd.concat([df, df2])
df3.shape

(7514, 4)

In [33]:
df3 = df3.reset_index()
df3.head()

,index,Unnamed: 0,Date,user_id,comments
0,0,0,2014-02-20 14:06:00,robr2,Someone better call marketing then:
1,1,1,2014-02-20 14:14:00,andys120,All what? This whole renaming scheme has turne...
2,2,2,2014-02-20 14:34:00,flightnurse,LOL... Oh well....
3,3,3,2014-02-20 15:04:00,Michaell,"Isn't there going to be a 4-series ""grand coup..."
4,4,4,2014-02-20 15:13:00,kyfdx,l


In [36]:
df3 = df3[['Date', 'user_id', 'comments']]

In [34]:
len(df3['user_id'].value_counts())

130

In [37]:
df3.to_csv("7500_comments_edmunds.csv", sep="\t")

In [79]:
df['Date'].min()

Timestamp('2014-02-20 14:06:00')

#### Start with Sentence